In [81]:
# Create Training DataSet and Model Testing Dataset of Results DataFrame 

import pandas
import numpy as np 
import matplotlib.pyplot as plt  
from pandas.api.types import is_numeric_dtype
from pandas import read_csv 

race_results = read_csv('results_full3(+tyrestrat_weather).csv')
del race_results['name_x.1']

In [82]:
for col in race_results.columns:
    #if not is_numeric_dtype(race_results[col]):
        race_results[col] = race_results[col].astype('category') 
        race_results[col] = race_results[col].cat.codes

In [83]:
#splitting the dataset into the Predictors (independant variables) and the Result variable (dependant variable)
predictors = race_results.drop('positionOrder', axis =1) 
predictors = race_results.drop('date', axis =1)#all columns except the results column
PositionOrder = race_results[['positionOrder']] #only the Position column

In [84]:
#running Recursive Feature Elimination to get top 5 variables for use in KNN model 

# Compute Information Gain of each feauture and select only features with GINI greater than 70 

from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
from sklearn import datasets

svm = LinearSVC()
# create the RFE model for the svm classifier 
# and select attributes
rfe = RFE(svm, 28)
rfe = rfe.fit(predictors,PositionOrder)
# print summaries for the selection of attributes
print(rfe.support_)
print(rfe.ranking_)

[False  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True False  True  True  True  True  True  True  True
  True  True  True False  True  True  True  True]
[5 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1]


In [86]:
# feature extraction using extrstreesclassifier
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(predictors,PositionOrder)
print(model.feature_importances_)

[ 0.01400617  0.01984881  0.01392545  0.01608003  0.01642407  0.03022192
  0.16423899  0.23954767  0.17440092  0.01946091  0.01509898  0.01493706
  0.01977901  0.0151516   0.01672678  0.01579187  0.01209024  0.01368006
  0.0136595   0.01237581  0.01425861  0.01270126  0.01679438  0.01149876
  0.02419733  0.00092787  0.00174857  0.00662908  0.01385462  0.01164798
  0.01038139  0.01791428]


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [88]:
# Spliting Data into 60% Training and 40% Testing 
from sklearn.model_selection import train_test_split
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(predictors,PositionOrder, test_size=0.4) 


#trainingSet, testingSet = train_test_split(race_results, test_size=0.2)

In [89]:
# Run Build KNN model and predict testing data 
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


In [90]:
# Test KNN Model using Testing Dataset 
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

('Accuracy:', 0.042091836734693876)


In [92]:
# Build Confusion Matrix of predicted results 
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 6, 12,  6,  3,  4,  1,  0,  0,  1,  1,  1,  0,  0,  2,  1,  0,  0,
         0,  0,  0,  0,  0],
       [15,  6,  6,  1,  6,  0,  2,  1,  2,  2,  1,  0,  1,  1,  1,  0,  0,
         0,  0,  0,  0,  0],
       [11,  7,  4,  2,  6,  1,  2,  2,  1,  0,  2,  1,  0,  0,  0,  0,  0,
         0,  1,  0,  0,  0],
       [17,  6,  8,  3,  9,  3,  2,  1,  1,  4,  0,  0,  0,  1,  0,  0,  0,
         0,  1,  0,  0,  0],
       [ 7,  5,  6,  1,  2,  0,  1,  1,  2,  1,  1,  1,  2,  2,  1,  0,  0,
         1,  1,  0,  0,  0],
       [ 6,  7,  2,  0, 10,  2,  3,  3,  2,  0,  2,  3,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [11,  5,  4,  1,  6,  4,  2,  2,  3,  2,  0,  1,  0,  1,  0,  1,  0,
         0,  1,  0,  0,  0],
       [ 9,  3,  6,  2,  4,  1,  4,  1,  3,  1,  1,  1,  0,  0,  0,  0,  1,
         1,  0,  0,  0,  0],
       [ 9,  3,  9,  3,  5,  1,  5,  1,  0,  1,  0,  2,  1,  2,  0,  0,  1,
         0,  0,  1,  0,  0],
       [ 7,  2,  5,  1,  4,  2,  6,  2,  0,  1,  0,  0,